In [1]:
import json
import os
import pandas as pd

folder_path = r"C:\Users\ihsiu\Desktop\exa-data-eng-assessment-main\data\sample"

def process_json_files(folder):
    all_key_value_data = []  # List to store individual JSON key-value data

    for filename in os.listdir(folder):
        if filename.endswith(".json"):
            file_path = os.path.join(folder, filename)
            with open(file_path, "r") as json_file:
                data = json.load(json_file)

                entry_data = data.get("entry", [])
                for entry in entry_data:
                    resource_data = entry["resource"]

                    # Extracting key-value pairs from resource
                    entry_key_value_data = [{"Key": key, "Value": value} for key, value in resource_data.items()]
                    all_key_value_data.extend(entry_key_value_data)

                    # Extracting fullUrl
                    full_url = entry.get("fullUrl", None)
                    all_key_value_data.append({"Key": "fullUrl", "Value": full_url})

                    # Extracting nested key-value pairs from request
                    request_data = entry.get("request", {})
                    for key, value in request_data.items():
                        if isinstance(value, dict):
                            for nested_key, nested_value in value.items():
                                all_key_value_data.append({"Key": f"request.{key}.{nested_key}", "Value": nested_value})
                        else:
                            all_key_value_data.append({"Key": f"request.{key}", "Value": value})

    # Create a DataFrame from the combined key-value data
    key_value_df = pd.DataFrame(all_key_value_data)

    return key_value_df


In [2]:
# Call the function with your folder path
key_value_df = process_json_files(folder_path)
# pd.set_option('display.max_rows', None)

key_value_df



,Key,Value
0,resourceType,Patient
1,id,cced3031-d98c-d870-5dce-f0086d8c7a34
2,meta,{'profile': ['http://hl7.org/fhir/us/core/Stru...
3,text,"{'status': 'generated', 'div': '<div xmlns=""ht..."
4,extension,[{'url': 'http://hl7.org/fhir/us/core/Structur...
...,...,...
37496,recorded,2021-09-30T06:06:44.325+01:00
37497,agent,[{'type': {'coding': [{'system': 'http://termi...
37498,fullUrl,urn:uuid:7e44ff49-8fe6-f6af-0dc4-c78bad96b917
37499,request.method,POST


In [3]:
df = key_value_df[key_value_df['Key'] == 'resourceType']
df

,Key,Value
0,resourceType,Patient
17,resourceType,Encounter
32,resourceType,Condition
46,resourceType,DiagnosticReport
61,resourceType,DocumentReference
...,...,...
37419,resourceType,DiagnosticReport
37434,resourceType,DocumentReference
37450,resourceType,Claim
37468,resourceType,ExplanationOfBenefit


In [4]:
df.groupby('Value').nunique()


,Key
Value,
CarePlan,1
CareTeam,1
Claim,1
Condition,1
Device,1
DiagnosticReport,1
DocumentReference,1
Encounter,1
ExplanationOfBenefit,1


In [39]:
import os
import pandas as pd
import json
import psycopg2

def read_nested_json(file_path):
    with open(file_path, 'r', encoding="utf8") as file:
        data = json.load(file)
    return data

def normalize_nested_json(data):
    # Use pandas json_normalize() to flatten the nested JSON data
    df = pd.json_normalize(data, record_path=['entry'], max_level=2, sep='_')
    return df

def process_json_files(json_directory):
    
    all_dataframes = []

    # Iterate through all files in the directory
    for filename in os.listdir(json_directory):
        if filename.endswith(".json"):
            file_path = os.path.join(json_directory, filename)
            nested_json_data = read_nested_json(file_path)
            normalized_df = normalize_nested_json(nested_json_data)
            all_dataframes.append(normalized_df)

    # Concatenate all dataframes into a single dataframe
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df

def create_resource_dataframes(combined_df):
    
    resourceType_lst = combined_df['resource_resourceType'].unique().tolist()

    # Create an empty dictionary to store the dataframes
    dataframes_dict = {}

    # Loop through each resource type and create the respective dataframe
    for rT in resourceType_lst:
        # Filter the rows for the current resource type and drop columns with NaN values
        df = combined_df.loc[combined_df['resource_resourceType'] == rT].dropna(axis=1).convert_dtypes()

        # Store the dataframe in the dictionary with the resource type as the key
        dataframes_dict[rT] = df

    return dataframes_dict
        

if __name__ == "__main__":
    
    json_directory = r"C:\Users\ihsiu\Desktop\exa-data-eng-assessment-main\data"  # Replace this with the path to your directory containing JSON files
    
    combined_df = process_json_files(json_directory)
    dataframes_dict = create_resource_dataframes(combined_df)

    # Now you can access each dataframe using its corresponding resource type as the key
    # For example:
    df_encounter = dataframes_dict['Encounter']
    df_patient = dataframes_dict['Patient']
    df_condition = dataframes_dict['Condition']
    df_diagrep = dataframes_dict['DiagnosticReport']
    df_explan = dataframes_dict['ExplanationOfBenefit']
    df_medreq = dataframes_dict['MedicationRequest']
    df_careteam = dataframes_dict['CareTeam']
    df_careplan = dataframes_dict['CarePlan']
    df_procedure = dataframes_dict['Procedure']
    df_immun = dataframes_dict['Immunization']
    df_observ = dataframes_dict['Observation']
    df_proven = dataframes_dict['Provenance']
    df_device = dataframes_dict['Device']


In [40]:
combined_df

,fullUrl,resource_resourceType,resource_id,resource_meta_profile,resource_text_status,resource_text_div,resource_extension,resource_identifier,resource_name,resource_telecom,...,resource_lotNumber,resource_serialNumber,resource_deviceName,resource_type_text,resource_hospitalization_dischargeDisposition,resource_multipleBirthInteger,resource_suppliedItem_quantity,resource_suppliedItem_itemCodeableConcept,resource_dosage_dose,resource_dosage_rateQuantity
0,urn:uuid:8c95253e-8ee8-9ae8-6d40-021d702dc78e,Patient,8c95253e-8ee8-9ae8-6d40-021d702dc78e,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Dickens475', '...","[{'system': 'phone', 'value': '555-152-6034', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,urn:uuid:4dbc90e0-b7b2-482c-24af-1405654e59ae,Encounter,4dbc90e0-b7b2-482c-24af-1405654e59ae,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,"[{'use': 'official', 'system': 'https://github...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,urn:uuid:2b9b5daf-ef78-63c7-3edc-cfc0a209eee0,Condition,2b9b5daf-ef78-63c7-3edc-cfc0a209eee0,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,urn:uuid:b6487994-9b3f-7b3c-6e2d-feba134129f9,Condition,b6487994-9b3f-7b3c-6e2d-feba134129f9,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,urn:uuid:f76535a7-ac06-3e3c-d0a3-7285fb4c8dd8,DiagnosticReport,f76535a7-ac06-3e3c-d0a3-7285fb4c8dd8,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63559,urn:uuid:b9a774c6-9582-de86-3593-2460084a0533,DiagnosticReport,b9a774c6-9582-de86-3593-2460084a0533,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63560,urn:uuid:71993cc6-3ebc-1e08-0b30-4e518294d124,DocumentReference,71993cc6-3ebc-1e08-0b30-4e518294d124,[http://hl7.org/fhir/us/core/StructureDefiniti...,NaN,NaN,NaN,"[{'system': 'urn:ietf:rfc:3986', 'value': 'urn...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63561,urn:uuid:33d4ef7d-71ce-82b3-e5d3-01d7bd52aa1a,Claim,33d4ef7d-71ce-82b3-e5d3-01d7bd52aa1a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63562,urn:uuid:60e0a7dd-16e1-6b8f-3203-3dc18f1b5f15,ExplanationOfBenefit,60e0a7dd-16e1-6b8f-3203-3dc18f1b5f15,NaN,NaN,NaN,NaN,[{'system': 'https://bluebutton.cms.gov/resour...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df_patient

,fullUrl,resource_resourceType,resource_id,resource_meta_profile,resource_text_status,resource_text_div,resource_extension,resource_identifier,resource_name,resource_telecom,resource_gender,resource_birthDate,resource_address,resource_maritalStatus_coding,resource_maritalStatus_text,resource_communication,request_method,request_url
0,urn:uuid:8c95253e-8ee8-9ae8-6d40-021d702dc78e,Patient,8c95253e-8ee8-9ae8-6d40-021d702dc78e,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Dickens475', '...","[{'system': 'phone', 'value': '555-152-6034', ...",male,1944-08-28,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,M,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
619,urn:uuid:6fa23508-960e-ff22-c3d0-0519a036543b,Patient,6fa23508-960e-ff22-c3d0-0519a036543b,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Jerde200', 'gi...","[{'system': 'phone', 'value': '555-455-2751', ...",male,2001-06-28,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,Never Married,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
1552,urn:uuid:83524678-9bff-93b7-ef89-d7f5390072ff,Patient,83524678-9bff-93b7-ef89-d7f5390072ff,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Price929', 'gi...","[{'system': 'phone', 'value': '555-106-9705', ...",female,1980-06-10,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,S,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
2255,urn:uuid:4e343b0a-8698-b6dd-64c6-c2d2d0959e6e,Patient,4e343b0a-8698-b6dd-64c6-c2d2d0959e6e,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Hansen121', 'g...","[{'system': 'phone', 'value': '555-675-9219', ...",female,1946-05-21,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,M,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
3216,urn:uuid:6e4ac285-2a8d-a30d-5ecb-e32cb595a876,Patient,6e4ac285-2a8d-a30d-5ecb-e32cb595a876,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Anderson154', ...","[{'system': 'phone', 'value': '555-418-5997', ...",female,1968-07-07,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,M,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60292,urn:uuid:15201e77-7e36-0667-73bc-013783e57649,Patient,15201e77-7e36-0667-73bc-013783e57649,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Hodkiewicz467'...","[{'system': 'phone', 'value': '555-165-4080', ...",female,1946-02-17,[{'extension': [{'url': 'http://hl7.org/fhir/S...,[{'system': 'http://terminology.hl7.org/CodeSy...,M,[{'language': {'coding': [{'system': 'urn:ietf...,POST,Patient
616

In [47]:
df_immun.dtypes

fullUrl                          string
resource_resourceType            string
resource_id                      string
resource_meta_profile            object
request_method                   string
request_url                      string
resource_status                  string
resource_encounter_reference     string
resource_patient_reference       string
resource_location_reference      string
resource_location_display        string
resource_vaccineCode_coding      object
resource_vaccineCode_text        string
resource_occurrenceDateTime      string
resource_primarySource          boolean
dtype: object